# 1.2 Neural network basics

In this notebook, we construct a basic neural network in PyTorch. We attempt to only provide a bare-bone example for clarity. For another such example, please see [PyTorch's homepage](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

<!--
- [Building the network](#Building-the-network)
- [Training the network](#Training-the-network)
- [Testing the network](#Testing-the-network)
- [Exercises](#Exercises)
-->

First, we import the libraries and load the data. You have already seen this code in notebook 1.1.

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"


In [ ]:

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# We load a training set
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# And withhold some data for testing
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True) # Torch's default dataloader is sequential and slow. Use num_worker may speed up the process a bit, but still CPU-driven. Write own loader if data fits on GPU
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=256, shuffle=False)

## Building the network

Below, we construct a fully connected neural network. Here, we will limit ourselves to a single hidden layer, i.e., it's a shallow rather than deep neural network. To construct a neural network in PyTorch, we define a class that inherits from the nn.Module class. 

The class contains two functions:

1) init: Defines all the objects we will need.

2) forward: puts all of these objects together, defining the network architecture. Basically, x starts out as your input tensor and is transformed step by step as it passes through each layer of the network. In this notebook, we consider MNIST images as our input tensors. Each image contains 28x28 pixels. The fully connected layers of the neural network, however, can only deal with one-dimensional feature vectors. So, we flatten the tensor, x, which is then passed on to the first fully connected layer, fc1. After the first fully connected layer, a Rectified Linear Unit (ReLU) activation function is applied element-wise to the tensor. The tensor is then passed through the second fully connected layer, fc2. After passing through the second fully connected layer, the transformed x is returned. The final tensor (x) represents the output of the neural network, which can be used for tasks like classification (you can rename x if you don't want to call the input and the output the same or if you want to distinguish between feature extraction and the final classification).

In [17]:
# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

You can now create an instance of the class.

In [18]:
model = SimpleNet()
model = model.to(device)

## Training the network

We now want to train the weights of the neural network. For this purpose, we first need to define what a good fit to data entails.

In [19]:
criterion = nn.CrossEntropyLoss().to(device)

Secondly, we need to decide how we want to optimise the network.

In [20]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

Now, we are ready to train the neural network. Let's train for 5 epochs. Note that fixing the number of epochs in this manner is not a good choice and will lead to under- or overfitting. But in this notebook, we merely want to understand the basic concepts behind training a neural network.

In each epoch, for each batch from the training set, we

1) clear the accumulated gradients of the model parameters (accumulated during backpropagation),

2) perform a forward run (i.e. we predict labels for the batch of images),

3) compute the loss for these predictions,

3) perform backpropagation,

5) and use this to optimise the weights and biases of our model.

In [ ]:
num_epochs = 5

# Training loop 
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    running_loss = 0.0
    # !!!!!!!!!!!!!!!!!!!!! IMPORTANT !!!!!!!!!!!!!!!!!!!!!
    for images, labels in train_loader:
        images = images.to(device, non_blocking=True) # SLOW!! # Move data from CPU to the GPU in parallel by num_workers in the DataLoader
        labels = labels.to(device, non_blocking=True) # SLOW!! # Move data from CPU to the GPU in parallel by num_workers in the DataLoader
        for param in model.parameters():
            param.grad = None
        # optimizer.zero_grad() # zero the parameter gradients
        # outputs = model(images) #.to("cpu") # forward run
        # loss = criterion(outputs, labels) # compare to ground truth
        with torch.autocast(device_type='cuda', dtype=torch.float32):
            outputs = model(images)
            loss = criterion(outputs, labels) 
        loss.backward() # back propagation
        optimizer.step() # update weights and biases
        
        running_loss += loss.item()
    # !!!!!!!!!!!!!!!!!!!!! IMPORTANT !!!!!!!!!!!!!!!!!!!!!
    # Print average loss for the epoch
    average_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss}')

Epoch [1/5], Loss: 0.16252010849879142
Epoch [2/5], Loss: 0.1526936502849802
Epoch [3/5], Loss: 0.144004159595104
Epoch [4/5], Loss: 0.14220418911030952
Epoch [5/5], Loss: 0.14520296237253127


## Testing the network

Having trained the network, we can now test it on unseen data (the test set).

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        

accuracy = correct / total
print(f'Accuracy on the test set: {100 * accuracy}%')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

That's a pretty high accuracy, reflecting that the MNIST dataset is relatively simple and well-behaved. Indeed, you can do much better. CNNs can reach an accuracy of more than 99 per cent (see [Kaggle](https://www.kaggle.com/code/cdeotte/how-to-choose-cnn-architecture-mnist)). To achieve such a high accuracy, we need to improve the architecture. We will do so in notebook 2.1.

## Exercises

**Exercise 1**: Here, we use the adam optimiser. Find out what other optimisers are available.

**Exercise 2**: What do the hyperparameters (e.g. lr in adam) mean? Change lr to a higher value (e.g. 1) and rerun the notebook. Or to a lower value. What happens? Why?

**Exercise 3**: Here, we use the cross entropy as the loss function. What is a loss function? And what is the cross entropy?

**Exercise 4**: Integrate data augmentation techniques into the data preprocessing pipeline. This helps improve the model's generalisation.

**Exercise 5**: Change the architecture of the neural network. Add more layers, increase the number of neurons, or try a different activation function. 